In [1]:
# # python3 /home/rnb/Projects/rnb-planning/src/pkg/planning/filtering/lattice_model/shared_lattice_predictor.py --rtype panda

# import SharedArray as sa
# ROBOT_TYPE_NAME = 'panda'
# sa.delete("shm://{}.prepared".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.grasp_img".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.arm_img".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.rh_vals".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.result".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.query_in".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.response_out".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.query_quit".format(ROBOT_TYPE_NAME))

In [1]:
from __future__ import print_function
import os
RNB_PLANNING_DIR = os.environ["RNB_PLANNING_DIR"]
os.chdir(os.path.join(RNB_PLANNING_DIR, 'src'))
from pkg.controller.combined_robot import *
from pkg.utils.utils import get_now, try_mkdir
import subprocess

DATA_PATH = os.path.join(RNB_PLANNING_DIR, "data")
try_mkdir(DATA_PATH)

DAT_ROOT = "stowing-deep"
RES_ROOT = "stowing-deep-result-btf-ik08-to2.0"

TEST_DATA_PATH = os.path.join(DATA_PATH, DAT_ROOT)
try_mkdir(TEST_DATA_PATH)

TEST_RESULT_PATH = os.path.join(DATA_PATH, RES_ROOT)
try_mkdir(TEST_RESULT_PATH)

HEADS = ["plan_time", "length", "MP_count", "failed_MPs", "IK_count", "failed_IKs", "success"]

def run_test_loop(FILE_OPTION, I_START, I_END, common_args, CNAME_LIST=['None', 'Tool', 'ToolReach', 'Full']):
    N_DAT = I_END-I_START
    ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
    DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
    file_list = sorted(os.listdir(DATASET_PATH))

    gtimer = GlobalTimer.instance()
    gtimer.reset(scale=1, timeunit='s')

    gtimer.tic("full_loop")
    for fid in range(I_START, I_END):
        e_step_list = []
        for i_c, cname in enumerate(CNAME_LIST):
            gtimer.tic("step")
            p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                                  '--data_idx', str(fid), '--file_option', FILE_OPTION, '--cname', cname,]
                                 + common_args)
            p.wait()
            e_step = np.round(gtimer.toc("step"), 1)
            e_step_list.append(e_step)
            print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")

        print("")
        print("========== {} / {} (eta: {} / {} s) =========".format(
            fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid-I_START+1, N_DAT), 1)))

## Common params

In [2]:
RTYPE = "panda"
DAT_DIR = '20210917-113211'
I_START = 0
I_END = 50


CNAME_LIST = ['None', 'Tool', 'ToolReach', 'Full']
# CNAME_LIST = ['Full'] #,'Pairwise']
VISUALIZE = False
PLAY_RESULT = False
SHOW_STATE = False

TIMEOUT_MOTION = 2.0
MAX_TIME = 100
MAX_ITER = 100

GRASP_SAMPLE = 30
STABLE_SAMPLE = 50
SEARCH_SAMPLE_RATIO= 100
MAX_SKELETONS = 30
TIMED_COMPLETE = False
STACK_TIMELOG = True
IK_TRY_NUM = 8
VERBOSE = False
USE_MOVEIT_IK = True


common_args = [
            '--rtype', RTYPE,
            '--dat_root', DAT_ROOT,
            '--res_root', RES_ROOT,
            '--dat_dir', DAT_DIR,
            '--VISUALIZE', str(VISUALIZE),
            '--PLAY_RESULT', str(PLAY_RESULT),
            '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
            '--MAX_TIME', str(MAX_TIME),
            '--MAX_ITER', str(MAX_ITER), 
            '--GRASP_SAMPLE', str(GRASP_SAMPLE),
            '--STABLE_SAMPLE', str(STABLE_SAMPLE),
            '--SEARCH_SAMPLE_RATIO', str(SEARCH_SAMPLE_RATIO),
            '--MAX_SKELETONS', str(MAX_SKELETONS),
            '--TIMED_COMPLETE', str(TIMED_COMPLETE),
            '--STACK_TIMELOG', str(STACK_TIMELOG),
            '--IK_TRY_NUM', str(IK_TRY_NUM),
            '--VERBOSE', str(VERBOSE),
            '--USE_MOVEIT_IK', str(USE_MOVEIT_IK)]

# Test all

In [3]:
FILE_OPTION = "obj_1c_obs0"
run_test_loop(FILE_OPTION, I_START, I_END, common_args, CNAME_LIST=CNAME_LIST)

FILE_OPTION = "obj_1c_obs3"
run_test_loop(FILE_OPTION, I_START, I_END, common_args, CNAME_LIST=CNAME_LIST)

FILE_OPTION = "obj_1c_obs5"
run_test_loop(FILE_OPTION, I_START, I_END, common_args, CNAME_LIST=CNAME_LIST)

FILE_OPTION = "obj_1c_obs7"
run_test_loop(FILE_OPTION, I_START, I_END, common_args, CNAME_LIST=CNAME_LIST)

FILE_OPTION = "obj_3c_obs3"
run_test_loop(FILE_OPTION, I_START, I_END, common_args, CNAME_LIST=CNAME_LIST)

FILE_OPTION = "obj_3c_obs5"
run_test_loop(FILE_OPTION, I_START, I_END, common_args, CNAME_LIST=CNAME_LIST)

FILE_OPTION = "obj_3c_obs7"
run_test_loop(FILE_OPTION, I_START, I_END, common_args, CNAME_LIST=CNAME_LIST)

------- 11.0    10.7    8.7     9.3     -------
========== 1 / 50 (eta: 39.8 / 1987.7 s) =========
------- 13.4    8.3     8.7     8.9     -------
========== 2 / 50 (eta: 79.1 / 1978.5 s) =========
------- 8.4     8.5     14.2    12.4    -------
========== 3 / 50 (eta: 122.7 / 2044.3 s) =========
------- 12.5    33.2    12.7    8.9     -------
========== 4 / 50 (eta: 190.0 / 2375.0 s) =========
------- 8.9     8.4     9.6     9.2     -------
========== 5 / 50 (eta: 226.1 / 2261.4 s) =========
------- 14.7    8.8     9.4     13.3    -------
========== 6 / 50 (eta: 272.4 / 2269.9 s) =========
------- 8.2     8.1     8.6     8.7     -------
========== 7 / 50 (eta: 306.0 / 2185.8 s) =========
------- 10.1    8.7     8.6     8.7     -------
========== 8 / 50 (eta: 342.2 / 2138.5 s) =========
------- 10.9    10.3    11.1    10.8    -------
========== 9 / 50 (eta: 385.3 / 2140.7 s) =========
------- 9.0     12.4    9.8     9.5     -------
========== 10 / 50 (eta: 426.0 / 2130.2 s) =========
-

------- 8.9     9.4     10.8    14.3    -------
========== 32 / 50 (eta: 1666.9 / 2604.6 s) =========
------- 11.3    9.5     21.4    11.2    -------
========== 33 / 50 (eta: 1720.4 / 2606.7 s) =========
------- 15.8    8.5     11.0    9.6     -------
========== 34 / 50 (eta: 1765.4 / 2596.2 s) =========
------- 8.2     8.5     8.7     8.8     -------
========== 35 / 50 (eta: 1799.7 / 2571.0 s) =========
------- 9.9     9.2     10.7    21.5    -------
========== 36 / 50 (eta: 1851.1 / 2570.9 s) =========
------- 10.8    14.4    9.8     8.9     -------
========== 37 / 50 (eta: 1894.9 / 2560.7 s) =========
------- 9.1     13.0    8.9     9.6     -------
========== 38 / 50 (eta: 1935.6 / 2546.8 s) =========
------- 9.1     8.3     10.9    9.8     -------
========== 39 / 50 (eta: 1973.8 / 2530.5 s) =========
------- 11.3    8.9     11.1    10.4    -------
========== 40 / 50 (eta: 2015.5 / 2519.4 s) =========
------- 11.3    10.1    10.9    8.8     -------
========== 41 / 50 (eta: 2056.7 / 

------- 8.9     9.3     9.3     9.0     -------
========== 13 / 50 (eta: 573.8 / 2207.0 s) =========
------- 9.1     9.1     11.1    11.1    -------
========== 14 / 50 (eta: 614.2 / 2193.7 s) =========
------- 20.0    16.4    16.0    12.8    -------
========== 15 / 50 (eta: 679.5 / 2265.0 s) =========
------- 17.4    10.6    16.3    9.9     -------
========== 16 / 50 (eta: 733.7 / 2292.9 s) =========
------- 10.9    10.0    11.9    9.8     -------
========== 17 / 50 (eta: 776.4 / 2283.6 s) =========
------- 8.5     8.4     9.4     8.9     -------
========== 18 / 50 (eta: 811.7 / 2254.7 s) =========
------- 9.3     11.8    13.5    10.0    -------
========== 19 / 50 (eta: 856.4 / 2253.6 s) =========
------- 10.3    15.6    8.8     13.1    -------
========== 20 / 50 (eta: 904.3 / 2260.7 s) =========
------- 12.6    14.4    13.5    12.2    -------
========== 21 / 50 (eta: 957.1 / 2278.7 s) =========
------- 108.1   17.0    11.8    46.9    -------
========== 22 / 50 (eta: 1140.9 / 2593.0 s)

------- 30.4    18.7    22.5    12.8    -------
========== 44 / 50 (eta: 3243.4 / 3685.7 s) =========
------- 16.5    15.6    15.6    9.6     -------
========== 45 / 50 (eta: 3300.8 / 3667.6 s) =========
------- 18.3    14.3    13.0    9.2     -------
========== 46 / 50 (eta: 3355.8 / 3647.6 s) =========
------- 10.6    11.1    12.5    10.2    -------
========== 47 / 50 (eta: 3400.2 / 3617.3 s) =========
------- 10.3    10.3    13.2    10.0    -------
========== 48 / 50 (eta: 3444.0 / 3587.5 s) =========
------- 9.0     15.0    11.2    17.5    -------
========== 49 / 50 (eta: 3496.8 / 3568.2 s) =========
------- 11.4    11.3    9.1     9.6     -------
========== 50 / 50 (eta: 3538.3 / 3538.3 s) =========
------- 9.7     9.1     14.1    10.0    -------
========== 1 / 50 (eta: 43.0 / 2150.5 s) =========
------- 9.9     16.8    9.6     20.0    -------
========== 2 / 50 (eta: 99.4 / 2484.8 s) =========
------- 12.4    13.1    9.9     10.7    -------
========== 3 / 50 (eta: 145.6 / 2426.7 s

------- 10.0    9.6     9.6     9.5     -------
========== 25 / 50 (eta: 2676.2 / 5352.4 s) =========
------- 14.9    11.4    11.8    13.3    -------
========== 26 / 50 (eta: 2727.7 / 5245.5 s) =========
------- 32.7    81.5    25.1    19.8    -------
========== 27 / 50 (eta: 2886.8 / 5346.0 s) =========
------- 11.1    12.0    10.2    14.4    -------
========== 28 / 50 (eta: 2934.5 / 5240.3 s) =========
------- 12.0    30.3    31.7    14.2    -------
========== 29 / 50 (eta: 3022.8 / 5211.8 s) =========
------- 15.2    11.5    12.1    14.6    -------
========== 30 / 50 (eta: 3076.3 / 5127.2 s) =========
------- 14.2    9.0     11.2    9.9     -------
========== 31 / 50 (eta: 3120.7 / 5033.4 s) =========
------- 10.9    13.4    11.8    10.0    -------
========== 32 / 50 (eta: 3166.9 / 4948.3 s) =========
------- 9.7     12.9    10.9    11.1    -------
========== 33 / 50 (eta: 3211.6 / 4866.1 s) =========
------- 14.3    12.0    11.3    9.5     -------
========== 34 / 50 (eta: 3258.8 / 